<a href="https://colab.research.google.com/github/ryota-sb/py-colaboratory/blob/main/first_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

kira2 = music_pb2.NoteSequence()

pg = 4
is_drum = False

kira2.notes.add(pitch=70, start_time=0.0, end_time=0.4, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=70, start_time=0.4, end_time=0.8, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=77, start_time=0.8, end_time=1.2, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=77, start_time=1.2, end_time=1.6, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=79, start_time=1.6, end_time=2.0, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=79, start_time=2.0, end_time=2.4, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=77, start_time=2.4, end_time=3.2, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=75, start_time=3.2, end_time=3.6, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=75, start_time=3.6, end_time=4.0, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=74, start_time=4.0, end_time=4.4, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=74, start_time=4.4, end_time=4.8, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=72, start_time=4.8, end_time=5.2, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=72, start_time=5.2, end_time=5.6, velocity=80, program=pg, is_drum=is_drum)
kira2.notes.add(pitch=70, start_time=5.6, end_time=6.4, velocity=80, program=pg, is_drum=is_drum)

kira2.total_time =6.4
kira2.tempos.add(qpm=150)

note_seq.plot_sequence(kira2)
note_seq.play_sequence(kira2, synth=note_seq.fluidsynth)

In [ ]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(kira2, "kira2.mid") # midiデータに変換保存
files.download("kira2.mid") # ファイルダウンロード

# 機械学習のモデルについて
モデルとは、定量的なルールを数式で表したもの  
モデルは、多数のパラメータ、ハイパーパラメータを値や設定として持つ

# Melody RNN
再帰型ニューラルネットワーク（RNN）の一種、LSTMを用いたメロディ生成する

# Music VAE
VAE（変換オートエンコーダ）を用いた音楽生成する

# GANSynth
GAN（敵対的生成ネットワーク）を用いてオーディオを合成する

In [ ]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle

note_seq.notebook_utils.download_bundle("basic_rnn.mag", "/models/")
bundle = sequence_generator_bundle.read_bundle_file("/models/basic_rnn.mag")
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map["basic_rnn"](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

In [ ]:
from note_seq.protobuf import generator_pb2

base_sequence = kira2
total_time = 36
temperature = 1.2

base_end_time = max(note.end_time for note in base_sequence.notes)

generator_options = generator_pb2.GeneratorOptions()
generator_options.args["temperature"].float_value = temperature
generator_options.generate_sections.add(
    start_time=base_end_time,
    end_time=total_time)

gen_seq = melody_rnn.generate(base_sequence, generator_options)

note_seq.plot_sequence(gen_seq)
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)

In [18]:
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

In [ ]:
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

music_vae = TrainedModel(
    configs.CONFIG_MAP["cat-mel_2bar_big"],
    batch_size=4,
    checkpoint_dir_or_path="/content/mel_2bar_big.ckpt")

In [ ]:
import note_seq

generated = music_vae.sample(n=5,
                             length=128,
                             temperature=1.0)

for ns in generated:
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

In [ ]:
# 最初のNoteSequence
import magenta
import note_seq
from note_seq.protobuf import music_pb2

kira2_start = music_pb2.NoteSequence()

kira2_start.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
kira2_start.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80)
kira2_start.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80)
kira2_start.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
kira2_start.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
kira2_start.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80)
kira2_start.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80)
kira2_start.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80)
kira2_start.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80)
kira2_start.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80)
kira2_start.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80)
kira2_start.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80)
kira2_start.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
kira2_start.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80)

kira2_start.total_time =6.4
kira2_start.tempos.add(qpm=75)

note_seq.plot_sequence(kira2_start)
note_seq.play_sequence(kira2_start, synth=note_seq.fluidsynth)

# 最後のNoteSequence
kira2_end = music_pb2.NoteSequence()

kira2_end.notes.add(pitch=70, start_time=0.0, end_time=0.4, velocity=80)
kira2_end.notes.add(pitch=72, start_time=0.4, end_time=0.8, velocity=80)
kira2_end.notes.add(pitch=74, start_time=0.8, end_time=1.2, velocity=80)
kira2_end.notes.add(pitch=77, start_time=1.2, end_time=1.6, velocity=80)
kira2_end.notes.add(pitch=79, start_time=1.6, end_time=2.0, velocity=80)
kira2_end.notes.add(pitch=74, start_time=2.0, end_time=2.4, velocity=80)
kira2_end.notes.add(pitch=70, start_time=2.4, end_time=3.2, velocity=80)
kira2_end.notes.add(pitch=72, start_time=3.2, end_time=3.6, velocity=80)
kira2_end.notes.add(pitch=74, start_time=3.6, end_time=4.0, velocity=80)
kira2_end.notes.add(pitch=77, start_time=4.0, end_time=4.4, velocity=80)
kira2_end.notes.add(pitch=79, start_time=4.4, end_time=4.8, velocity=80)
kira2_end.notes.add(pitch=74, start_time=4.8, end_time=5.2, velocity=80)
kira2_end.notes.add(pitch=72, start_time=5.2, end_time=5.6, velocity=80)
kira2_end.notes.add(pitch=70, start_time=5.6, end_time=6.4, velocity=80)

kira2_end.total_time =6.4
kira2_end.tempos.add(qpm=75)

note_seq.plot_sequence(kira2_end)
note_seq.play_sequence(kira2_end, synth=note_seq.fluidsynth)

In [ ]:
n_seq = 8 # NoteSequenceの数

gen_seq = music_vae.interpolate(
    kira2_start,
    kira2_end,
    num_steps=n_seq,
    length=32)

interp_seq = note_seq.sequences_lib.concatenate_sequences(gen_seq)

note_seq.plot_sequence(interp_seq)
note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)